In [281]:
!git clone https://github.com/AshishJangra27/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [282]:
import pandas as pd
data=pd.read_csv('/content/datasets/Job Postings/jobs.csv.zip')
data.head()

,job_id,job_role,company,experience,salary,location,rating,reviews,resposibilities,posted_on,job_link,company_link
0,7.012301e+10,Branch Banking - Calling For Women Candidates,Hdfc Bank,1-6 Yrs,Not disclosed,"Kolkata, Hyderabad/Secunderabad, Pune, Ahmedab...",4.0,39110 Reviews,"Customer Service,Sales,Relationship Management",1 Day Ago,https://www.naukri.com/job-listings-branch-ban...,https://www.naukri.com/hdfc-bank-jobs-careers-213
1,6.012391e+10,Product Owner Senior Manager,Accenture,11-15 Yrs,Not disclosed,"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",4.1,32129 Reviews,"Product management,Market analysis,Change mana...",1 Day Ago,https://www.naukri.com/job-listings-product-ow...,https://www.naukri.com/accenture-jobs-careers-...
2,6.012391e+10,Employee Relations and Policies Associate Manager,Accenture,3-7 Yrs,Not disclosed,"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",4.1,32129 Reviews,"Business process,Change management,Team manage...",1 Day Ago,https://www.naukri.com/job-listings-employee-r...,https://www.naukri.com/accenture-jobs-careers-...
3,6.012391e+10,Employee Relations and Policies Specialist,Accenture,3-7 Yrs,Not disclosed,"Kolkata, Mumbai, Hyderabad/Secunderabad, Pune,...",4.1,32129 Reviews,"Business process,Change management,Team manage...",1 Day Ago,https://www.naukri.com/job-listings-employee-r...,https://www.naukri.com/accenture-jobs-careers-...
4,6.012301e+10,SAP BO Consultant,Mindtree,5-7 Yrs,Not disclosed,"Hybrid - Kolkata, Hyderabad/Secunderabad, Pune...",4.1,3759 Reviews,"SAP BO,PL / SQL,Oracle SQL,SAP Business Object...",1 Day Ago,https://www.naukri.com/job-listings-sap-bo-con...,https://www.naukri.com/mindtree-jobs-careers-3...


**PROBLEM STATEMENT: location based job search**

# **data exploration**
- REMOVE UNNECSECARY COLUMNS
- DEALING WITH NULL VALUES
- DEALING WITH DUPLICATE VALUES

In [283]:
del data['posted_on']
del data['job_link']
del data['company_link']

In [284]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79147 entries, 0 to 79146
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   job_id           78667 non-null  float64
 1   job_role         78667 non-null  object 
 2   company          78666 non-null  object 
 3   experience       77398 non-null  object 
 4   salary           78667 non-null  object 
 5   location         77441 non-null  object 
 6   rating           42948 non-null  float64
 7   reviews          42948 non-null  object 
 8   resposibilities  78647 non-null  object 
dtypes: float64(2), object(7)
memory usage: 5.4+ MB


In [285]:
data.isnull().sum()

,0
job_id,480
job_role,480
company,481
experience,1749
salary,480
location,1706
rating,36199
reviews,36199
resposibilities,500


In [286]:
data.dropna(subset=['company', 'job_id', 'resposibilities'], inplace=True)
data['experience']=data['experience'].fillna('5-10 years')#filling null with the most common values
data['location']=data['location'].fillna('Bangalore/Bengaluru')
#no. of missing rating and reviews is same means they are new companies
data['rating']=data['rating'].fillna('0.0')
data['reviews']=data['reviews'].fillna('0 Reviews')
data.isnull().sum()

,0
job_id,0
job_role,0
company,0
experience,0
salary,0
location,0
rating,0
reviews,0
resposibilities,0


In [287]:
data.duplicated().sum()

np.int64(5629)

In [288]:
data.drop_duplicates(subset='job_id').duplicated().sum()

np.int64(0)

**change data type of job id column**

In [289]:
data['job_id']=data['job_id'].astype('int64').astype(str)

**clean experience column**

In [290]:
data['experience'].value_counts(10)

,proportion
experience,
5-10 Yrs,0.102218
3-8 Yrs,0.056583
1-6 Yrs,0.041986
4-9 Yrs,0.040511
2-7 Yrs,0.039125
...,...
20-26 Yrs,0.000013
22-28 Yrs,0.000013
19-24 Yrs,0.000013


In [291]:
#minimum expereince column
data['min_experience']=data['experience'].str.replace('Yrs','').str.replace('years','').str.split('-').str[0].str.strip().astype('int')

In [292]:
data['max_experience']=data['experience'].str.replace('Yrs','').str.replace('years','').str.split('-').str[1].str.strip().astype('int')

**cleaning salary and location column**

In [293]:
#print(data['location'])

data['clean_location']=data['location'].str.replace('(All Areas)','').str.replace(',','-').str.replace('/','~').str.strip().str.strip().str.split('~')

In [294]:
data['salary']

,salary
0,Not disclosed
1,Not disclosed
2,Not disclosed
3,Not disclosed
4,Not disclosed
...,...
79142,"7,00,000 - 17,00,000 PA."
79143,Not disclosed
79144,Not disclosed
79145,"2,50,000 - 3,50,000 PA."


In [295]:
import re

def clean_and_convert_salary(salary_str):
    salary_str = str(salary_str).strip()
    if 'not disclosed' in salary_str.lower():
        return 0

    # Handle crore values
    if 'cr' in salary_str.lower():
        # Extract the number before 'Cr'
        crore_value = re.search(r'(\d+\.?\d*)', salary_str)
        if crore_value:
            return int(float(crore_value.group(1)) * 10000000)

    # Handle Lakhs and other formats
    numbers = re.findall(r'\d+\.?\d*', salary_str.replace(',', ''))
    numbers = [float(num) for num in numbers]

    if not numbers:
        return 0

    # If it's a range, return the average, otherwise the single value
    if len(numbers) > 1:
        return int((numbers[0] + numbers[1]) / 2 * 100000) # Assuming lakhs if not specified
    else:
        # Check if it's likely lakhs
        if numbers[0] < 100: # Heuristic for lakhs
             return int(numbers[0] * 100000)
        return int(numbers[0])



def get_min_max_salary(salary_str, get_max=False):
    salary_str = str(salary_str).strip()
    if 'not disclosed' in salary_str.lower():
        return 0

    # Handle crore values
    if 'cr' in salary_str.lower():
        # Extract the number before 'Cr'
        crore_value = re.search(r'(\d+\.?\d*)', salary_str)
        if crore_value:
            val = int(float(crore_value.group(1)) * 10000000)
            return val

    # Handle Lakhs and other formats
    numbers = re.findall(r'\d[\d,.]*', salary_str)

    if not numbers:
        return 0

    cleaned_numbers = [int(float(n.replace(',', ''))) for n in numbers]

    if 'lakh' in salary_str.lower() or (all(i<100 for i in cleaned_numbers) and len(cleaned_numbers)>0): # Heuristic for lakhs
      cleaned_numbers = [i * 100000 for i in cleaned_numbers]


    if get_max and len(cleaned_numbers)>1:
      return cleaned_numbers[1]

    return cleaned_numbers[0]


data['min_salary'] = data['salary'].apply(lambda x: get_min_max_salary(x))
data['max_salary'] = data['salary'].apply(lambda x: get_min_max_salary(x, get_max=True))

In [297]:
data.tail()

,job_id,job_role,company,experience,salary,location,rating,reviews,resposibilities,min_experience,max_experience,clean_location,min_salary,max_salary
79142,60123006615,.Net Fullstack Developer,Citiustech,3-8 Yrs,"7,00,000 - 17,00,000 PA.","Hybrid - Pune, Bangalore/Bengaluru, Mumbai (Al...",4.0,742 Reviews,"c#,.net core,javascript,Microservices,Angular,...",3,8,"[Hybrid - Pune- Bangalore, Bengaluru- Mumbai]",700000,1700000
79143,11222009693,Senior (BI) Business Intelligence Architect,Hexaware Technologies,10-13 Yrs,Not disclosed,"Hybrid - Pune, Chennai, Bangalore/Bengaluru, M...",3.7,3172 Reviews,"Informatica Mdm,etl,business intelligence,Agil...",10,13,"[Hybrid - Pune- Chennai- Bangalore, Bengaluru-...",0,0
79144,151222006599,PowerBI Developer,EY,5-10 Yrs,Not disclosed,"Hybrid - Pune, Mumbai (All Areas)",3.8,6267 Reviews,"pagination,Power Bi,Dax",5,10,[Hybrid - Pune- Mumbai],0,0
79145,271222005611,Customer Service Associate,Tech Mahindra,0-2 Yrs,"2,50,000 - 3,50,000 PA.","Temp. WFH - Noida, Kolkata, New Delhi, Hyderab...",3.7,21440 Reviews,"Bengali,International BPO,BPO,Customer Service...",0,2,[Temp. WFH - Noida- Kolkata- New Delhi- Hydera...,250000,350000
79146,211222006941,Senior Java BED ( PAN India)- KG,Tech Mahindra,8-12 Yrs,"25,00,000 - 27,50,000 PA.","Temp. WFH - Noida, Kolkata, Hyderabad/Secunder...",3.7,21440 Reviews,"Kafka,sql,spring boot,REST API,java,Lambda,Des...",8,12,"[Temp. WFH - Noida- Kolkata- Hyderabad, Secund...",2500000,2750000
